In [ ]:
# Standard installs
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets # If loading from Hugging Face Hub

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mt9vwavh/unsloth_f9b86b3a97d149a29efcbedfce35820b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mt9vwavh/unsloth_f9b86b3a97d149a29efcbedfce35820b
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 11.8 MB/s eta 0:00:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Chat Template for Classification (Sentiment Analysis)

In [ ]:
# CORRECT MODEL from Huggingface directly
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # Correct path
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


# Enable LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

# Load a small classification dataset
dataset = load_dataset("glue", "sst2", split = "train[:500]")  # Only 500 samples

# Format dataset into a chat prompt
def formatting_prompts_func(example):
    text = example['sentence']
    label = example['label']
    label_text = "positive" if label == 1 else "negative"

    prompt = f"<|im_start|>system\nYou are a helpful assistant classifying sentiment.<|im_end|>\n<|im_start|>user\nClassify the sentiment of this sentence: {text}<|im_end|>\n<|im_start|>assistant\n{label_text}<|im_end|>"
    return {"text": prompt}

dataset = dataset.map(formatting_prompts_func)

# Tokenization
def tokenize_function(sample):
    model_inputs = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=2048,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(["sentence", "label", "text"])

# Training Preparation
FastLanguageModel.for_training(model)

from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model = model,
    train_dataset = dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 30,  # Small demo
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs_chat_template_classification",
        optim = "paged_adamw_32bit",
        fp16 = True,
        bf16 = False,
    ),
)

trainer.train()

# Save final model
model.save_pretrained("tinyllama_classification_unsloth")
tokenizer.save_pretrained("tinyllama_classification_unsloth")

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.1 patched 22 layers with 22 QKV layers, 22 O layers and 0 MLP layers.


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayanth-kalyanam (jayanth-kalyanam-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,13.275700
2,13.237100
3,13.041700
4,13.157800
5,13.462500
6,13.289500
7,13.157000
8,13.129600
9,11.651200
10,9.305500


('tinyllama_classification_unsloth/tokenizer_config.json',
 'tinyllama_classification_unsloth/special_tokens_map.json',
 'tinyllama_classification_unsloth/tokenizer.model',
 'tinyllama_classification_unsloth/added_tokens.json',
 'tinyllama_classification_unsloth/tokenizer.json')

## Conversational Chat Finetuning

In [ ]:
# CONTINUING after Task (c.1) model load

from datasets import load_dataset

# Load a small conversational dataset
dataset_conv = load_dataset("conv_ai_2", split="train[:1000]")  # Slightly bigger, but still lightweight

# CONTINUING after loading dataset_conv

# Correct formatting function
def formatting_prompts_conv(example):
    dialog = example['dialog']
    if len(dialog) < 2:
        return {"text": ""}  # Skip if not enough dialog turns

    user_message = dialog[-2]  # Second last message = User
    assistant_response = dialog[-1]  # Last message = Assistant

    prompt = f"<|im_start|>system\nYou are a helpful AI assistant.<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_response}<|im_end|>"
    return {"text": prompt}

dataset_conv = dataset_conv.map(formatting_prompts_conv)

# Continue with tokenization as before
def tokenize_function_conv(sample):
    model_inputs = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=2048,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset_conv = dataset_conv.map(tokenize_function_conv, batched=True)
dataset_conv = dataset_conv.remove_columns(["dialog", "text"])

# Training Preparation (REUSE model)
FastLanguageModel.for_training(model)

from transformers import Trainer, TrainingArguments

trainer_conv = Trainer(
    model = model,
    train_dataset = dataset_conv,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 40,  # Slightly longer
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs_conversational_chat",
        optim = "paged_adamw_32bit",
        fp16 = True,
        bf16 = False,
        save_strategy="no",  # To save memory, no checkpoint saving here
    ),
)

trainer_conv.train()

# Save after conversational finetuning
model.save_pretrained("tinyllama_conversational_unsloth")
tokenizer.save_pretrained("tinyllama_conversational_unsloth")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 2,252,800/4,000,000,000 (0.06% trained)


Step,Training Loss
1,4.691800
2,4.631700
3,4.626500
4,4.631800
5,4.580400
6,4.667400
7,4.581300
8,4.594500
9,4.622100
10,4.580600


('tinyllama_conversational_unsloth/tokenizer_config.json',
 'tinyllama_conversational_unsloth/special_tokens_map.json',
 'tinyllama_conversational_unsloth/tokenizer.model',
 'tinyllama_conversational_unsloth/added_tokens.json',
 'tinyllama_conversational_unsloth/tokenizer.json')

## Extending Maximum Context Size of TinyLlama

In [ ]:
# CONTINUING after conversational task

# Set tokenizer's model max length to safe limit
tokenizer.model_max_length = 2048

# Create dummy dataset with near-max length examples
dummy_texts = ["Hello " * 350, "This is a simulated long context. " * 350]

from datasets import Dataset
dataset_context = Dataset.from_list([{"text": text} for text in dummy_texts])

# Tokenization
def tokenize_long(sample):
    model_inputs = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=2048,  # NOT 4096
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset_context = dataset_context.map(tokenize_long, batched=True)

# Training Preparation
FastLanguageModel.for_training(model)

from transformers import Trainer, TrainingArguments

trainer_context = Trainer(
    model = model,
    train_dataset = dataset_context,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        max_steps = 20,
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs_extend_context",
        optim = "paged_adamw_32bit",
        fp16 = True,
        bf16 = False,
        save_strategy="no",
    ),
)

trainer_context.train()

# Save the model
model.save_pretrained("tinyllama_extended_context_unsloth")
tokenizer.save_pretrained("tinyllama_extended_context_unsloth")

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 10 | Total steps = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 2,252,800/4,000,000,000 (0.06% trained)


Step,Training Loss
1,8.272200
2,17.318200
3,8.272200
4,17.318200
5,8.272200
6,3.541000
7,3.541000
8,4.269300
9,0.026500
10,3.987200


('tinyllama_extended_context_unsloth/tokenizer_config.json',
 'tinyllama_extended_context_unsloth/special_tokens_map.json',
 'tinyllama_extended_context_unsloth/tokenizer.model',
 'tinyllama_extended_context_unsloth/added_tokens.json',
 'tinyllama_extended_context_unsloth/tokenizer.json')

## Multi-Dataset Single Finetuning

In [ ]:
# CONTINUING in same notebook, reusing model/tokenizer

from datasets import load_dataset, concatenate_datasets

# Load Classification Dataset (SST2 small subset)
dataset_classify = load_dataset("glue", "sst2", split="train[:500]")

def formatting_prompts_classify(example):
    text = example['sentence']
    label = example['label']
    label_text = "positive" if label == 1 else "negative"

    prompt = f"<|im_start|>system\nYou are a helpful assistant classifying sentiment.<|im_end|>\n<|im_start|>user\nClassify the sentiment of this sentence: {text}<|im_end|>\n<|im_start|>assistant\n{label_text}<|im_end|>"
    return {"text": prompt}

dataset_classify = dataset_classify.map(formatting_prompts_classify)
dataset_classify = dataset_classify.remove_columns(["sentence", "label"])

# Load Conversational Dataset (ConvAI2 small subset)
dataset_converse = load_dataset("conv_ai_2", split="train[:1000]")

def formatting_prompts_converse(example):
    dialog = example['dialog']
    if len(dialog) < 2:
        return {"text": ""}
    user_message = dialog[-2]
    assistant_response = dialog[-1]

    prompt = f"<|im_start|>system\nYou are a helpful AI assistant.<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_response}<|im_end|>"
    return {"text": prompt}

dataset_converse = dataset_converse.map(formatting_prompts_converse)
dataset_converse = dataset_converse.remove_columns(["dialog"])

# Merge both datasets
dataset_merged = concatenate_datasets([dataset_classify, dataset_converse])

# Tokenization
def tokenize_merged(sample):
    model_inputs = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=2048,
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

dataset_merged = dataset_merged.map(tokenize_merged, batched=True)
dataset_merged = dataset_merged.remove_columns(["text"])

# Training Preparation
FastLanguageModel.for_training(model)

from transformers import Trainer, TrainingArguments

trainer_merged = Trainer(
    model = model,
    train_dataset = dataset_merged,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 50,  # Slightly bigger since two tasks mixed
        learning_rate = 2e-4,
        logging_steps = 1,
        output_dir = "outputs_multi_task",
        optim = "paged_adamw_32bit",
        fp16 = True,
        bf16 = False,
        save_strategy="no",
    ),
)

trainer_merged.train()

# Save final model
model.save_pretrained("tinyllama_multi_task_unsloth")
tokenizer.save_pretrained("tinyllama_multi_task_unsloth")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 2,252,800/4,000,000,000 (0.06% trained)


Step,Training Loss
1,4.551000
2,4.572700
3,4.566100
4,4.579800
5,4.611600
6,4.526000
7,4.590700
8,4.522500
9,4.614400
10,4.537000


('tinyllama_multi_task_unsloth/tokenizer_config.json',
 'tinyllama_multi_task_unsloth/special_tokens_map.json',
 'tinyllama_multi_task_unsloth/tokenizer.model',
 'tinyllama_multi_task_unsloth/added_tokens.json',
 'tinyllama_multi_task_unsloth/tokenizer.json')